In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score

**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
train_df = pd.read_csv('kaggle_data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('kaggle_data/test_sessions.csv', index_col='session_id')

In [3]:
train_df[['site1', 'site2', 'site3', 'site4','site5', 'site6','site7', 'site8', 'site9', 'site10'] ].head(3)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,8,11.0,82.0,68.0,8.0,8393.0,8403.0,932.0,3260.0,8.0
2,111,78.0,151.0,111.0,1401.0,151.0,1375.0,38.0,1401.0,97.0
3,11,3187.0,132.0,496.0,1969.0,504.0,3187.0,82.0,3191.0,3184.0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [4]:
train_test_df = pd.concat([train_df, test_df])

In [5]:
print (train_df.shape, test_df.shape, train_test_df.shape)

(95319, 21) (41177, 20) (136496, 21)


In [6]:
train_test_df.iloc[95317:95321]

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time10,time2,time3,time4,time5,time6,time7,time8,time9,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
95318,223,173.0,466.0,3810.0,111.0,11.0,88.0,841.0,3895.0,111.0,...,2014-02-26 09:05:02,2014-02-26 09:05:00,2014-02-26 09:05:00,2014-02-26 09:05:00,2014-02-26 09:05:01,2014-02-26 09:05:01,2014-02-26 09:05:01,2014-02-26 09:05:02,2014-02-26 09:05:02,1273.0
95319,1943,1943.0,1943.0,1943.0,1943.0,1943.0,1943.0,1943.0,32.0,32.0,...,2014-10-04 10:17:59,2014-10-04 10:14:29,2014-10-04 10:14:59,2014-10-04 10:15:59,2014-10-04 10:16:30,2014-10-04 10:16:59,2014-10-04 10:17:29,2014-10-04 10:17:54,2014-10-04 10:17:55,3370.0
1,1917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20225,20225.0,27.0,9.0,32.0,20225.0,20258.0,20225.0,20225.0,20276.0,...,2014-03-04 14:11:09,2014-03-04 14:08:59,2014-03-04 14:08:59,2014-03-04 14:09:50,2014-03-04 14:10:43,2014-03-04 14:10:44,2014-03-04 14:10:45,2014-03-04 14:11:07,2014-03-04 14:11:08,NaN


In [7]:
test_df.sample(3)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
33766,2184,2014-01-04 21:36:12,2367.0,2014-01-04 21:36:28,2367.0,2014-01-04 21:36:32,2367.0,2014-01-04 21:36:33,2184.0,2014-01-04 21:36:37,2184.0,2014-01-04 21:36:54,2184.0,2014-01-04 21:36:55,15137.0,2014-01-04 21:36:55,16154.0,2014-01-04 21:36:56,15137.0,2014-01-04 21:36:58
36730,8,2014-05-02 13:49:54,421.0,2014-05-02 13:49:56,421.0,2014-05-02 13:49:57,869.0,2014-05-02 13:51:05,11420.0,2014-05-02 13:51:23,421.0,2014-05-02 13:51:24,252.0,2014-05-02 13:51:24,11420.0,2014-05-02 13:51:24,11420.0,2014-05-02 13:51:25,20.0,2014-05-02 13:51:33
14611,2,2013-11-12 08:20:50,58.0,2013-11-12 08:22:04,9.0,2013-11-12 08:25:05,9.0,2013-11-12 08:26:13,12.0,2013-11-12 08:26:38,8.0,2013-11-12 08:26:44,9.0,2013-11-12 08:27:19,12.0,2013-11-12 08:27:56,8.0,2013-11-12 08:27:56,9.0,2013-11-12 08:28:22


In [8]:
train_df['user_id'].nunique()

550

In [12]:
train_df['user_id'].value_counts().head(3)

3165    3351
537     1927
3324    1534
Name: user_id, dtype: int64

In [13]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [14]:
train_test_df_sites.head(3)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,8,11,82,68,8,8393,8403,932,3260,8
2,111,78,151,111,1401,151,1375,38,1401,97
3,11,3187,132,496,1969,504,3187,82,3191,3184


In [15]:
def sparse_array( sess, maxlen ) :
    
    indices = sess.ravel() 
    ln = len( indices )
    data = np.ones(ln, dtype=int)
    indptr = np.array( range(0, ln+1, maxlen), dtype=int )
    
    return csr_matrix( (data, indices, indptr),  dtype=int )[:, 1:] # первый столбец разреженной матрицы надо удаляем

In [16]:
X_train_test_sparse = sparse_array(train_test_df_sites.values, 10)

In [17]:
print (X_train_test_sparse.shape)

(136496, 24052)


In [18]:
N, _ = train_df.shape

In [19]:
X_train_sparse = X_train_test_sparse[:N, :]
X_test_sparse  = X_train_test_sparse[N:, :]
y = train_df.iloc[:, -1].values

In [20]:
n1, n2 = X_train_sparse.shape
n3, n4 = X_test_sparse.shape

In [21]:
print (str(n1)+" "+str(n2)+" "+str(n3)+" "+str(n4) )

95319 24052 41177 24052


**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [22]:
import pickle

with open('kaggle_data/X_train_sparse.pkl', 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl)
with open('kaggle_data/X_test_sparse.pkl', 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl)
with open('kaggle_data/train_target.pkl', 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl)

**Разобьем обучающую выборку на 2 части в пропорции 7/3.**

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=7, stratify=y)

In [25]:
sgd_logit = SGDClassifier(loss='log', random_state=7, n_jobs=-1 )

In [26]:
%%time
sgd_logit.fit(X_train, y_train)

Wall time: 11.5 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

In [27]:
sgd_svm = SGDClassifier(loss='hinge', random_state=7, n_jobs=-1 )

In [28]:
%%time
sgd_svm.fit(X_train, y_train)

Wall time: 9.43 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

**Сделаем прогнозы с помощью обеих моделей на отложенной выборке (X_valid, y_valid).**

In [29]:
logit_valid_pred = sgd_logit.predict(X_valid)
svm_valid_pred = sgd_svm.predict(X_valid)

In [30]:
print(accuracy_score(y_valid, logit_valid_pred))
print(accuracy_score(y_valid, svm_valid_pred))

0.286403692824
0.267135263673


In [31]:
logit_test_pred = sgd_logit.predict(X_test_sparse)

In [32]:
%%time
sgd_logit = SGDClassifier(loss='modified_huber', n_iter=123, penalty='l2', warm_start=True, random_state=7, n_jobs=-1 )
sgd_logit.fit(X_train, y_train)
logit_valid_pred = sgd_logit.predict(X_valid)
print(accuracy_score(y_valid, logit_valid_pred))  # baseline = 0.3123 loss='modified_huber', n_iter=101, penalty='l2'

0.315708490698
Wall time: 3min 58s


In [82]:
logit_test_pred = sgd_logit.predict(X_test_sparse)

In [1]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [83]:
write_to_submission_file(logit_test_pred, 'kaggle_data/sokolov_kaggle_submission.csv' )